In [1]:
import torch
from torch import Tensor, nn
from torch.types import Device, _size
from collections import OrderedDict

import os
import shutil
from pathlib import Path
import mne
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.preprocessing import Normalizer, StandardScaler
%matplotlib notebook

num_train = 2717
num_eval = 276

In [4]:
edf_folder = Path('../../Dataset/tuh_eeg_abnormal/')
channel_sums = None
channel_sums_squared = None
total_samples = 0
files = []
for file_path in edf_folder.glob('**/*.edf'):
    files.append(str(file_path))
print(files)

['../../Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaamsc_s001_t000.edf', '../../Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaajoo_s002_t001.edf', '../../Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaakoo_s001_t001.edf', '../../Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaakrt_s001_t001.edf', '../../Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaapqd_s001_t000.edf', '../../Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaakjk_s002_t002.edf', '../../Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaanyt_s002_t000.edf', '../../Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaaljf_s001_t000.edf', '../../Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaalyr_s001_t000.edf', '../../Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaampz_s001_t000.edf', '../../Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaalbh_s00

### Calculate mean and Standard deviation

In [5]:
edf_folder = Path('../../Dataset/tuh_eeg_abnormal/')
channel_sums = None
channel_sums_squared = None
total_samples = 0
for file_path in edf_folder.glob('**/*.edf'):
    raw = mne.io.read_raw_edf(file_path, preload=True)
    raw.crop(tmin=60)
    raw.resample(100)
    sfreq = raw.info['sfreq']
    data = raw.get_data(picks=['EEG FP1-REF', 'EEG FP2-REF', 'EEG F3-REF','EEG F4-REF', 
                               'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 
                               'EEG O1-REF', 'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF',
                               'EEG T3-REF', 'EEG T4-REF', 'EEG T5-REF', 'EEG T6-REF', 
                               'EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF'], 
                        return_times=False, units='uV')
    n_samples = data.shape[1]
    if channel_sums is None:
        n_channels = data.shape[0]
        channel_sums = np.zeros(n_channels)
        channel_sums_squared = np.zeros(n_channels)
    
    channel_sums += np.sum(data, axis=1)  # Sum of signal values
    channel_sums_squared += np.sum(data ** 2, axis=1)  # Sum of squared signal values
    total_samples += n_samples  # Accumulate the total number of samples
    
channel_means = channel_sums / total_samples
channel_vars = (channel_sums_squared / total_samples) - (channel_means ** 2)
channel_stds = np.sqrt(channel_vars)
for i in range(n_channels):
    print(f"Channel {i + 1}: Mean = {channel_means[i]}, Std = {channel_stds[i]}")

Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaamsc_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 314249  =      0.000 ...  1256.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaajoo_s002_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 300999  =      0.000 ...  1203.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaakoo_s001_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 310249  =      0.000 ...  1240.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaakrt_s001_t001.edf...
EDF file detected
Setting ch

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 307749  =      0.000 ...  1230.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaagvx_s002_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 304749  =      0.000 ...  1218.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaakrt_s002_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 294999  =      0.000 ...  1179.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaabsk_s007_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 357249  =      0.000 ...  1428.996 secs...
Extracting EDF parameters 

Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaapar_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 309749  =      0.000 ...  1238.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaaovo_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 374499  =      0.000 ...  1497.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaahte_s002_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 303999  =      0.000 ...  1215.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaamjc_s001_t001.edf...
EDF file detected
Setting ch

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 296749  =      0.000 ...  1186.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaajqk_s003_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 295499  =      0.000 ...  1181.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaabuv_s002_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 346249  =      0.000 ...  1384.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaaiue_s004_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 412249  =      0.000 ...  1648.996 secs...
Extracting EDF parameters 

Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaamft_s001_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 681499  =      0.000 ...  2725.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaamhk_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 292999  =      0.000 ...  1171.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaaosh_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 391999  =      0.000 ...  1567.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/abnormal/01_tcp_ar/aaaaanwc_s001_t001.edf...
EDF file detected
Setting ch

...

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 375499  =      0.000 ...  1501.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/train/normal/01_tcp_ar/aaaaactm_s002_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 328499  =      0.000 ...  1313.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/train/normal/01_tcp_ar/aaaaaisv_s003_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 294249  =      0.000 ...  1176.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/train/normal/01_tcp_ar/aaaaammn_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 309499  =      0.000 ...  1237.996 secs...
Extracting EDF parameters fro

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 301999  =      0.000 ...  1207.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/train/normal/01_tcp_ar/aaaaanet_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 331249  =      0.000 ...  1324.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/train/normal/01_tcp_ar/aaaaanch_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 345999  =      0.000 ...  1383.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/train/normal/01_tcp_ar/aaaaamqu_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 350249  =      0.000 ...  1400.996 secs...
Extracting EDF parameters fro

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 271999  =      0.000 ...  1087.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/train/normal/01_tcp_ar/aaaaakir_s001_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 296999  =      0.000 ...  1187.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/train/normal/01_tcp_ar/aaaaakqe_s001_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 313999  =      0.000 ...  1255.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/train/normal/01_tcp_ar/aaaaapsx_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 308499  =      0.000 ...  1233.996 secs...
Extracting EDF parameters fro

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 300499  =      0.000 ...  1201.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/train/normal/01_tcp_ar/aaaaapxy_s001_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 333249  =      0.000 ...  1332.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/train/normal/01_tcp_ar/aaaaaost_s001_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 263499  =      0.000 ...  1053.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/train/normal/01_tcp_ar/aaaaanmw_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 301749  =      0.000 ...  1206.996 secs...
Extracting EDF parameters fro

EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 298249  =      0.000 ...  1192.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/train/normal/01_tcp_ar/aaaaamms_s001_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 296999  =      0.000 ...  1187.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/train/normal/01_tcp_ar/aaaaaobv_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 331999  =      0.000 ...  1327.996 secs...
Extracting EDF parameters from /home/yossi/Projects/Dataset/tuh_eeg_abnormal/v3.0.1/edf/train/normal/01_tcp_ar/aaaaaojs_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 317249  =      0.000 ...  1268.996 secs...
Extracting EDF parameters fro

In [12]:
import argparse
import yaml

if __name__ == "__main__":
#     logger = logging.getLogger(__name__)  # Use the current module's name
#     logging.basicConfig(filename=f'./logs/{datetime.now().strftime("%y%m%d%H%M")}_resnet18.log', level=logging.DEBUG)
# #     logger.setLevel(logging.DEBUG)
#     handler = logging.StreamHandler()
#     # formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
#     # handler.setFormatter(formatter)
#     logger.addHandler(handler)
    parser = argparse.ArgumentParser()
    parser.add_argument("config_file", metavar="FILE", help="config file")
    # parser.add_argument('--run-dir', metavar='DIR', help='run directory')
    # parser.add_argument('--pdb', action='store_true', help='pdb')
    args = parser.parse_args(args=['configs/abnormal_12000.yml'])
    # args, opts = parser.parse_known_args()
    # f = 'configs/eeg_pt.yml'
    with open(args.config_file, 'r') as file:
        configs = yaml.safe_load(file)


In [16]:
configs['dataset']['std'] 

[131.48135945,
 106.25727984,
 124.03741234,
 92.54006106,
 121.32491175,
 91.33204798,
 124.02669847,
 98.37289956,
 121.19875962,
 94.25380354,
 124.27180189,
 108.16782282,
 116.69246245,
 91.69132988,
 120.04684075,
 90.20482523,
 122.40893776,
 119.17286241,
 122.06775861]

In [6]:
channel_means

array([-0.84925363, -0.57970241, -0.84873335, -0.65943293, -0.42680256,
       -0.55692595, -0.68915349, -0.76090215, -0.81714699, -0.4305129 ,
       -0.90823503, -0.66544529, -1.0178982 , -0.65606002, -1.12267558,
       -0.55692305, -0.27457596, -0.47549457,  0.16019251])

In [7]:
channel_stds

array([131.48135945, 106.25727984, 124.03741234,  92.54006106,
       121.32491175,  91.33204798, 124.02669847,  98.37289956,
       121.19875962,  94.25380354, 124.27180189, 108.16782282,
       116.69246245,  91.69132988, 120.04684075,  90.20482523,
       122.40893776, 119.17286241, 122.06775861])

In [ ]:
import os
os.cpu_count()

### Read edf

In [ ]:
file_path = '../data/edf/normal/aaaaacby_s004_t000.edf'
raw = mne.io.read_raw_edf(file_path)
raw.resample(100)    # resampling to xHz
sfreq = raw.info['sfreq']   # 100
#     logger.info(freq)
raw.crop(tmin=60)

In [ ]:
pd_frame = raw.to_data_frame(picks=['EEG FP1-REF', 'EEG FP2-REF', 'EEG F3-REF','EEG F4-REF', 
                                            'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 
                                            'EEG O1-REF', 'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF',
                                            'EEG T3-REF', 'EEG T4-REF', 'EEG T5-REF', 'EEG T6-REF', 
                                            'EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF'])

In [ ]:
raw.info

In [ ]:
raw_cp = raw.copy().pick('EEG FP1-REF')
raw_cp.crop(tmin=0, tmax=5.0)
raw_cp.plot(scalings=dict(eeg=40e-6))

### Save to csv

In [ ]:
pd_frame.to_csv('../test.csv', index=False)

### Read csv

In [ ]:
# df = pd.read_csv('../data/seg_csv/aaaaacby_s004_t000_13.csv')
df = pd.read_csv('../test.csv')

In [ ]:
df.shape

In [ ]:
# Create a figure and a set of subplots
fig, ax = plt.subplots()  # Adjust the grid size as needed
ax = ax.plot(df.iloc[1000:1500,1])
# ax.set_title(f'Channel {df.iloc[0,1]}')
# ax.set_xlabel('Sample Index')
# ax.set_ylabel('Signal Value')
plt.show()

In [ ]:
scaler = Normalizer()
segment = scaler.fit_transform(df.T)
segment = segment.T

In [ ]:
fig, ax = plt.subplots()  # Adjust the grid size as needed
ax = ax.plot(segment[:,0])
# ax.set_title(f'Channel {df.iloc[0,1]}')
# ax.set_xlabel('Sample Index')
# ax.set_ylabel('Signal Value')
plt.show()

In [ ]:
scaler_1 = StandardScaler()
segment_1 = scaler_1.fit_transform(df.T)
segment_1 = segment.T

fig, ax = plt.subplots()  # Adjust the grid size as needed
ax = ax.plot(segment[:,0])
# ax.set_title(f'Channel {df.iloc[0,1]}')
# ax.set_xlabel('Sample Index')
# ax.set_ylabel('Signal Value')
plt.show()

In [ ]:
# Create a figure and a set of subplots
fig, axes = plt.subplots(nrows=5, ncols=4, figsize=(30, 20))  # Adjust the grid size as needed

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plot each channel in a separate subplot
for i in range(19):
    ax = axes[i]
    ax.plot(df.iloc[:, i])
    ax.set_title(f'Channel {i+1}')
    ax.set_xlabel('Sample Index')
    ax.set_ylabel('Signal Value')

# Remove the unused subplot (if the number of subplots is more than the channels)
for j in range(19, len(axes)):
    fig.delaxes(axes[j])

# Adjust the layout to prevent overlapping
plt.tight_layout()

# Display the plot
plt.show()



In [ ]:
df.iloc[500:600, 2]

In [ ]:
raw.plot()

In [ ]:
data_len = 6000
raw.resample(100)    # resampling to xHz
sfreq = raw.info['sfreq']   # 100
#     logger.info(freq)
raw.crop(tmin=60)    # start from 60 secs

In [ ]:
n_segments = int(np.floor(raw.times[-1] *sfreq/data_len))
pd_frame = raw.to_data_frame(picks=['EEG FP1-REF', 'EEG FP2-REF', 'EEG F3-REF','EEG F4-REF', 
                                            'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 
                                            'EEG O1-REF', 'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF',
                                            'EEG T3-REF', 'EEG T4-REF', 'EEG T5-REF', 'EEG T6-REF', 
                                            'EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF'])

In [ ]:
pd_frame.iloc[1,0]
segment = pd_frame.iloc[0:6000, 1:]
segment

In [ ]:
from sklearn.preprocessing import Normalizer
scaler = Normalizer()
seg_nor = scaler.fit_transform(segment)

In [ ]:
seg_nor[1800:2100,0]

### Data Processing

In [ ]:
import os
import shutil
from pathlib import Path
import mne
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.preprocessing import Normalizer

In [ ]:
edf_data_path = Path('./data/edf/train/')
edf_data_path.parts[-1]

In [ ]:
import logging

logger = logging.getLogger(__name__)  # Use the current module's name
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
# formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
# handler.setFormatter(formatter)
logger.addHandler(handler)
test_accuracy = 0.95  # Replace with your actual accuracy calculation
logger.info(f"test accuracy:{test_accuracy}")  # Log as info

In [ ]:
def positional_encoding(max_length:int, d_model:int, model_type='sinusoidal'):
    """
    Generates positional encodings for a given maximum sequence length and model dimensionality.

    Args:
        max_length (int): The maximum length of the sequence.
        d_model (int): The dimensionality of the model.
        model_type (str): The type of positional encoding to use. Defaults to 'sinusoidal'.

    Returns:
        numpy.ndarray: The positional encoding matrix of shape (max_length, d_model).
    """

    if model_type == 'sinusoidal':
        pe = np.zeros((max_length, d_model))
        position = np.arange(0, max_length, dtype=np.float32).reshape(-1, 1)
        div_term = np.exp(np.arange(0, d_model, 2) * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = np.sin(position * div_term)
        if pe.size % 2 != 0:
            pe[:, 1::2] = np.cos(position[:-1] * div_term)
    else:
        raise ValueError("Unsupported model_type: {}".format(model_type))

    return pe

# pe_train = positional_encoding(X_train.shape[0], X_train.shape[1])
# pe_test = positional_encoding(X_test.shape[0], X_test.shape[1])
# # Add positional encoding to the signal
# X_train =  X_train + pe_train # Add corresponding row of pe matrix
# X_test =  X_test + pe_test

In [ ]:
def data_clip(data_path:Path, result_path:Path, data_len:int): # data_path = Path('./data/edf/train/'); 
                                                                # result_path = Path('./data/origin_csv/train/')
    if os.path.exists(result_dir_path):
        shutil.rmtree(result_dir_path)
    os.mkdir(result_dir_path)
    
    
    channels = ['Fp1', 'Fp2', 'F3','F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T3', 'T4',
                'T5', 'T6', 'Fz', 'Cz', 'Pz']
    label_path = data_path
    stage = str(data_path.parts[-1])
    label = pd.DataFrame(columns=['csv_file','label'])
#     pe = positional_encoding(data_len, len(channels))    # 1000 needs to adjust according my study
    
    
    for file_path in data_path.glob('**/*.edf'):
        sub_label = str(file_path.parts[3])
        file_name = str(file_path.name).split('.')[0]
    #     print(sub_label, file_path, file_name)
        raw = mne.io.read_raw_edf(file_path)
        raw.resample(100)    # resampling to xHz
        sfreq = raw.info['sfreq']   # 100
    #     logger.info(freq)
        raw.crop(tmin=60)    # start from 60 secs
#         n_segments = int(np.floor(raw.times[-1] *sfreq/data_len))
        start, end = 0, data_len   # initilize slide window
        count = 0  # initilize num.of segments
        
        pd_frame = raw.to_data_frame(picks=['EEG FP1-REF', 'EEG FP2-REF', 'EEG F3-REF','EEG F4-REF', 
                                            'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 
                                            'EEG O1-REF', 'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF',
                                            'EEG T3-REF', 'EEG T4-REF', 'EEG T5-REF', 'EEG T6-REF', 
                                            'EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF'])
    #     channels = ['EEG FP1-REF', 'EEG FP2-REF', 'EEG F3-REF','EEG F4-REF', 
    #                                         'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 
    #                                         'EEG O1-REF', 'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF',
    #                                         'EEG T3-REF', 'EEG T4-REF', 'EEG T5-REF', 'EEG T6-REF', 
    #                                         'EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF']    
#         print(pd_frame.shape)
    #     logger.debug(pe)  # Log as info
        while end <= pd_frame.shape[0]:
#         for i in range(n_segments):
#             start_time = i * data_len  # Start time of the segment in seconds
#             end_time = start_time + data_len  # End time of the segment in seconds

            # Extract the segment
            segment = pd_frame.iloc[start:end, 1:]
    #         logger.info(segment.shape, file_name)

#             scaler = Normalizer()
#             segment = scaler.fit_transform(segment.T)
#             segment = segment.T + pe
#             np.savetxt(f'{str(result_dir_path)}/{file_name}_{count+1}.csv', segment, header=','.join(channels), 
#                        delimiter=',')
            segment.to_csv(f'{str(result_dir_path)}/{file_name}_{count+1}.csv', index=False)

            label.loc[len(label)] = [f'{file_name}_{count+1}.csv', sub_label]
            start += data_len
            end += data_len
            count += 1
        
        raw.close()
    label.to_csv(f'../data/{stage}_label_{segment_length}.csv', index=False)
    

### Process training dataset

In [ ]:
# training dataset
# edf_data_path = Path('../../Dataset/tuh_eeg_abnormal/v3.0.1/edf/train/')  # need to modify
edf_data_path = Path('../data/edf/')
result_dir_path = Path('../data/seg_csv/train_12000')
segment_length = 12000
data_clip(edf_data_path, result_dir_path, segment_length)

# edf_data_path = Path('../../Dataset/tuh_eeg_abnormal/v3.0.1/edf/eval/')  # need to modify
# result_dir_path = Path('./data/seg_csv/eval_12000/')
# data_clip(edf_data_path, result_dir_path, segment_length)

In [ ]:
scaler = Normalizer()
segment = scaler.fit_transform(segment.T)
#             segment = segment + pe
#         if segment.shape[0]>1000
segment = pd.DataFrame(segment.T, columns=channels)

#         data, times = raw[:, start_time*freq:end_time*freq]
#     print(times)
#         df = raw.to_data_frame()
segment.to_csv(f'{str(result_dir_path)}/{file_name}_{i+1}.csv', index=False)
#         np.savetxt(f'./data/origin_csv/train/{file_name}_{i+1}.csv', segment, header=','.join(channels), 
#                    delimiter=',')
label.loc[len(label)] = [f'{file_name}_{i+1}.csv', sub_label]

#### Dataset